In [ ]:
from content.layout_lm_tutorial.layoutlm_preprocess import *
import pytesseract

In [ ]:
from PIL import Image

In [ ]:
image = Image.open("your folder path/content/data/testing_data/images/invoice_250-page-1.jpg")
image = image.convert("RGB")
# image

In [ ]:
from torch.nn import CrossEntropyLoss

def get_labels(path):
    with open(path, "r") as f:
        labels = f.read().splitlines()
    if "O" not in labels:
        labels = ["O"] + labels
    return labels

labels = get_labels("your folder path/content/data/labels.txt")
num_labels = len(labels)
label_map = {i: label for i, label in enumerate(labels)}
# Use cross entropy ignore index as padding label id so that only real label ids contribute to the loss later
# pad_token_label_id = CrossEntropyLoss().ignore_index

In [ ]:
model_path='your folder path/content/layoutlm_invoices_May6_100epoch.pt'
model=model_load(model_path,num_labels)
image, words, boxes, actual_boxes = preprocess("your folder path/content/data/testing_data/images/invoice_250-page-1.jpg")

word_level_predictions, final_boxes=convert_to_features(image, words, boxes, actual_boxes, model)

In [ ]:
invoice_labels = set([item[2:] for item in label_map.values() if len(item) > 1])

In [ ]:
invoice_labels

In [ ]:
final_predictions = {item: set() for item in invoice_labels}

In [ ]:
image_path = 'your folder path/content/data/testing_data/images/invoice_250-page-1.jpg'
image = cv2.imread(image_path)
print(type(image))

image = Image.open(image_path)
print(type(image))

In [ ]:
import pytesseract
import cv2
image_path = 'your folder path/content/data/testing_data/images/invoice_250-page-1.jpg'
image = cv2.imread(image_path)
# Open the image
# image = Image.open(image_path)
# draw = ImageDraw.Draw(image)
# font = ImageFont.load_default()
def iob_to_label(label):
  if label != 'O':
    return label[2:]
  else:
    return ""
# label2color = {'invoice_number':'blue', 'invoice_date':'green', 'client_name':'coral', 'client_address':'hotpink', 'client_tax_id': 'springgreen', 'seller_name': 'black', 'seller_address': 'blueviolet', 'seller_tax_id': 'brown', 'iban': 'turquoise'}
for prediction, box in zip(word_level_predictions, final_boxes):
    predicted_label = iob_to_label(label_map[prediction]).upper()
    if predicted_label:
        x1, y1, x2, y2 = box
        # Crop image to the rectangle
        cropped_image = image[y1-10:y2+10, x1-10:x2+10]

        # Perform OCR on the cropped region
        text = pytesseract.image_to_string(cropped_image)
        
        # Print the extracted text
        if len(text)>2:
            print(predicted_label, text)
            final_predictions[predicted_label].add(text.strip('\n\x0c'))


In [ ]:
final_predictions

In [ ]:
image_path = 'your folder path/content/data/testing_data/images/invoice_250-page-1.jpg'
# Open the image
image = Image.open(image_path)
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()
def iob_to_label(label):
  if label != 'O':
    return label[2:]
  else:
    return ""
label2color = {'invoice_number':'blue', 'invoice_date':'green', 'client_name':'coral', 'client_address':'hotpink', 'client_tax_id': 'springgreen', 'seller_name': 'black', 'seller_address': 'blueviolet', 'seller_tax_id': 'brown', 'iban': 'turquoise'}
for prediction, box in zip(word_level_predictions, final_boxes):
    predicted_label = iob_to_label(label_map[prediction]).lower()
    if predicted_label:
      draw.rectangle(box, outline=label2color[predicted_label])
      draw.text((box[0] + 10, box[1] - 10), text=predicted_label, fill=label2color[predicted_label], font=font)
image